# Qwen4-TTS Voice Clone POC
## シンプルな音声クローニング実装

3秒程度の参照音声から声をクローンして、任意のテキストを読み上げます。

## 1. セットアップ

In [1]:
# 必要なライブラリのインストール
!pip install -q qwen-tts soundfile torch torchaudio

import torch
import soundfile as sf
import numpy as np
from IPython.display import Audio, display
from qwen_tts import Qwen3TTSModel
import os
import time
from concurrent.futures import ThreadPoolExecutor, TimeoutError

# デバイス設定
device = "cuda:0" if torch.cuda.is_available() else "cpu"
dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

print(f"✓ デバイス: {device}")
print(f"✓ データ型: {dtype}")

^C


ImportError: DLL load failed while importing _C: The specified module could not be found.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.0+cpu requires torch==2.1.0, but you have torch 2.10.0 which is incompatible.
xformers 0.0.33.post2 requires torch==2.9.1, but you have torch 2.10.0 which is incompatible.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\dance\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


## 2. モデルのロード

In [ ]:
# Qwen3-TTSモデルの初期化
print("モデルをロード中...")
model = Qwen3TTSModel.from_pretrained(
    "Qwen/Qwen3-TTS-0.5B",
    device=device,
    dtype=dtype
)
print("✓ モデルロード完了")

## 3. ファイルパスの設定

In [ ]:
# Google Driveのマウント（Colab環境の場合）
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base_path = '/content/drive/MyDrive'
    print("✓ Google Drive マウント完了")
except:
    # ローカル環境の場合
    base_path = '.'
    print("✓ ローカル環境で実行")

# ファイルパス設定
reference_audio = "001.wav"  # 参照音声ファイル
output_dir = os.path.join(base_path, "tts_outputs")
os.makedirs(output_dir, exist_ok=True)

print(f"参照音声: {reference_audio}")
print(f"出力先: {output_dir}")

## 4. 参照音声の確認

In [ ]:
# 参照音声ファイルの存在確認と情報表示
if os.path.exists(reference_audio):
    audio_data, sr = sf.read(reference_audio)
    duration = len(audio_data) / sr
    print(f"✓ 参照音声ファイル確認完了")
    print(f"  - サンプリングレート: {sr} Hz")
    print(f"  - 長さ: {duration:.2f} 秒")
    print(f"  - チャンネル数: {audio_data.ndim}")
    
    # 参照音声を再生
    display(Audio(reference_audio))
else:
    print(f"❌ エラー: {reference_audio} が見つかりません")
    print("ファイルパスを確認してください")

## 5. タイムアウト機能付き音声生成関数

In [ ]:
def generate_with_timeout(model, text, prompt_audio_path, speed=1.0, timeout=600):
    """
    タイムアウト機能付きの音声生成関数
    
    Args:
        model: Qwen3TTSModel インスタンス
        text: 生成するテキスト
        prompt_audio_path: 参照音声のパス
        speed: 再生速度（デフォルト: 1.0）
        timeout: タイムアウト時間（秒）（デフォルト: 600秒 = 10分）
    
    Returns:
        generated_audio: 生成された音声データ
    
    Raises:
        TimeoutError: 生成時間がタイムアウトを超えた場合
    """
    def _generate():
        return model.generate(
            text=text,
            prompt_audio_path=prompt_audio_path,
            speed=speed
        )
    
    with ThreadPoolExecutor(max_workers=1) as executor:
        future = executor.submit(_generate)
        try:
            result = future.result(timeout=timeout)
            return result
        except TimeoutError:
            print(f"❌ エラー: 音声生成が{timeout}秒を超えました")
            raise

print("✓ タイムアウト機能付き生成関数を定義")

## 6. 音声の長さチェック関数

In [ ]:
def check_audio_duration(audio_data, sample_rate, max_duration=60):
    """
    音声の長さをチェックする関数
    
    Args:
        audio_data: 音声データ（numpy配列）
        sample_rate: サンプリングレート
        max_duration: 最大長さ（秒）（デフォルト: 60秒）
    
    Returns:
        duration: 音声の長さ（秒）
        is_valid: 最大長さ以内かどうか
    """
    duration = len(audio_data) / sample_rate
    is_valid = duration <= max_duration
    
    if is_valid:
        print(f"✓ 音声長: {duration:.2f}秒 (制限: {max_duration}秒以内)")
    else:
        print(f"⚠ 警告: 音声長 {duration:.2f}秒が制限({max_duration}秒)を超えています")
    
    return duration, is_valid

print("✓ 音声長チェック関数を定義")

## 7. 音声生成の実行

In [ ]:
# 生成するテキスト（玉音放送の一節）
text = "耐え難きを耐え、忍び難きを忍び、もって万世のために太平を開かんと欲す"

print("="*60)
print("音声生成を開始します")
print("="*60)
print(f"テキスト: {text}")
print(f"参照音声: {reference_audio}")
print(f"タイムアウト: 10分")
print(f"最大音声長: 60秒")
print("="*60)

# 生成開始時刻を記録
start_time = time.time()

try:
    # タイムアウト付きで音声生成（10分 = 600秒）
    generated_audio = generate_with_timeout(
        model=model,
        text=text,
        prompt_audio_path=reference_audio,
        speed=1.0,
        timeout=600  # 10分のタイムアウト
    )
    
    # 生成時間を計算
    generation_time = time.time() - start_time
    
    print(f"\n✓ 音声生成完了")
    print(f"  生成時間: {generation_time:.2f}秒")
    
    # 音声の長さをチェック（60秒以内）
    duration, is_valid = check_audio_duration(
        generated_audio, 
        model.sample_rate, 
        max_duration=60
    )
    
    # ファイルに保存
    output_path = os.path.join(output_dir, "generated_gyokuon.wav")
    sf.write(output_path, generated_audio, model.sample_rate)
    print(f"\n✓ 保存完了: {output_path}")
    
    print("\n" + "="*60)
    print("生成結果サマリー")
    print("="*60)
    print(f"テキスト長: {len(text)}文字")
    print(f"生成時間: {generation_time:.2f}秒")
    print(f"音声長: {duration:.2f}秒")
    print(f"サンプリングレート: {model.sample_rate} Hz")
    print(f"ファイルサイズ: {os.path.getsize(output_path) / 1024:.2f} KB")
    print("="*60)
    
except TimeoutError:
    print("\n❌ 音声生成がタイムアウトしました（10分超過）")
    print("以下を確認してください:")
    print("  - テキストが長すぎないか")
    print("  - 参照音声の品質は適切か")
    print("  - GPUが正しく使用されているか")
    
except Exception as e:
    print(f"\n❌ エラーが発生しました: {str(e)}")
    import traceback
    traceback.print_exc()

## 8. 生成された音声の再生

In [ ]:
# 生成された音声を再生
if 'generated_audio' in globals():
    print("生成された音声:")
    display(Audio(generated_audio, rate=model.sample_rate))
    
    # 参照音声と比較用に並べて表示
    print("\n参照音声（比較用）:")
    display(Audio(reference_audio))
else:
    print("音声が生成されていません。上のセルを実行してください。")

## 9. 詳細な音声分析（オプション）

In [ ]:
# 音声の詳細分析（librosaを使用）
try:
    import librosa
    import librosa.display
    import matplotlib.pyplot as plt
    
    if 'generated_audio' in globals():
        # 波形表示
        plt.figure(figsize=(14, 5))
        plt.subplot(2, 1, 1)
        librosa.display.waveshow(generated_audio, sr=model.sample_rate)
        plt.title('生成音声の波形')
        plt.xlabel('時間 (秒)')
        plt.ylabel('振幅')
        
        # スペクトログラム表示
        plt.subplot(2, 1, 2)
        D = librosa.amplitude_to_db(np.abs(librosa.stft(generated_audio)), ref=np.max)
        librosa.display.specshow(D, sr=model.sample_rate, x_axis='time', y_axis='hz')
        plt.colorbar(format='%+2.0f dB')
        plt.title('スペクトログラム')
        
        plt.tight_layout()
        plt.show()
        
        # 統計情報
        print("\n音声統計情報:")
        print(f"  平均振幅: {np.mean(np.abs(generated_audio)):.6f}")
        print(f"  最大振幅: {np.max(np.abs(generated_audio)):.6f}")
        print(f"  RMS: {np.sqrt(np.mean(generated_audio**2)):.6f}")
    else:
        print("音声が生成されていません")
        
except ImportError:
    print("librosaがインストールされていません")
    print("詳細分析を行うには: !pip install librosa matplotlib")